# Основы программирования в Python

*Алла Тамбовцева, НИУ ВШЭ*


## Библиотека pandas. Продолжение.

### Группировка и агрегирование: методы `.groupby()` и `.agg()`

Часто случается, что данные необходимо сгруппировать по какому-то признаку – по значениям определенной переменной. На входе имеется таблица (датафрейм), а на выходе хочется получить несколько таблиц: отдельная таблица для каждого значения. Давайте рассмотрим такой пример. У нас есть база данных с результатами выборов, и нам нужно сгруппировать данные по регионам. 

Для начала импортируем библиотеку pandas и загрузим файл с данными.

In [ ]:
import pandas as pd

Для разнообразия загрузим файл по ссылке с Github:

In [ ]:
df = pd.read_csv("elect.csv")

В таблице сохранены результаты выборов президента России 2012 года. 

In [ ]:
df.head()

In [ ]:
df.info()

Таблица достаточно большая, поэтому давайте выберем те столбцы, которые понадобятся нам для работы. Какие именно? Столбцы в этой базе имеют порядковый номер строки в таблице на [сайте]() Центральной избирательной комиссии.

Выберем столбцы, которые соответствуют уровням комиссий, а также следующим показателям: общее число зарегистрированных избирателей, число недействительных бюллетеней, число действительных бюллетеней, число голосов за Жириновского, Зюганова, Миронова, Прохорова и Путина.

In [ ]:
d = df[["kom1", "kom2", "kom3", 
        "1", "9", "10", "19", 
        "20", "21", "22", "23"]]

In [ ]:
d.head()

Теперь присвоим столбцам более информативные названия:

In [ ]:
d.columns = ["region", "tik", "uik", 
             "total", "invalid", "valid", 
             "Zh", "Zu", "Mi", "Pr", "Pu"]

In [ ]:
d.head() # опять посмотрим

Посмотрим теперь, какие регионы есть в базе. Выбрать столбец *region* в таком случае будет не совсем удачно, поскольку в нем будет много повторяющихся значений. Посмотрим только на уникальные:

In [ ]:
d.region.unique() # метод unique - уникальные значения

Видно, что в этом массиве встречаются какие-то крокозябры (названия со странной кодировкой). Давайте уберем эти строки из базы. Воспользуемся методом `str.contains()` для строк:

In [ ]:
# отфильтруем с помощью условий
d = d[~d.region.str.contains("à")]  # ~ для отрицания

Сгруппируем данные по регионам и посчитаем для каждого региона явку в процентах и процент голосов за каждого кандидата. Группировка осуществляется с помощью метода `.groupby()`.

In [ ]:
d.groupby('region') # пока ничего не увидели

Что выдает метод `.groupby()`? На самом деле он делает следующее: создает список, состоящий из кортежей. Каждый кортеж – это пара *название группы*-*соответствующий ей фрагмент датафрейма*.

In [ ]:
# посмотрим на все сразу
for g in d.groupby('region'):
    print(g)

В таком виде метод `.groupby()` дает нам немного. Мы же хотим не просто получать отдельные таблицы, а агрегировать данные по регионам ‒ суммировать все показатели (число избирателей, бюллетеней, голосов) по каждому региону. Тут на помощь придет метод `.agg()`, который выполняет агрегирование по группам.

In [ ]:
d.groupby('region').agg('sum')

Сначала в `.groupby()` мы указали переменную, по которой нужно выполнить группировку, затем в `.agg()` мы указали функцию, которую нужно выполнить. В нашем случае это `'sum'`, поскольку нам нужно просто сложить все показатели в пределах одного региона. Применять можно и другие функции, например, считать среднее:

In [ ]:
d.groupby('region').agg('mean').head() # mean - среднее

Или сразу несколько статистик. которые можно указать в `.agg()` в виде списка.

In [ ]:
d.groupby('region').agg(['mean', 'median']).head() # среднее и медиана

Кроме того, внутри `.agg()` можно указывать свои функции. Например, нас интересует разница между максимальным и минимальным значением. Сначала напишем функцию `my_diff`, которая будет определять такую разность:

In [ ]:
def my_diff(x):
    return max(x) - min(x)

Проверим, как она работает:

In [ ]:
my_diff([4, 6, 8]) # все верно, 8 - 4 = 4

Теперь используем эту функцию внутри `.agg()`:

In [ ]:
d.groupby('region').agg(my_diff).head() # везде смотрим на первые 5 строк

Всё, что мы пока сделали, очень интересно, но есть проблема: все данные пока даны в абсолютных значениях, не в процентах. Это неудобно. Давайте сгруппируем данные по региону и добавим в базу с агрегированными данными новые столбцы: явка в процентах и проценты голосов за каждого кандидата.

Для этого необходимо вспомнить, как считается явка и проценты голосов. Явка считается так: суммируем число действительных и недействительных бюллетеней. Чтобы получить явку в процентах, делим явку на общее число зарегистрированных избирателей и домножаем на $100$, чтобы перевести долю в проценты. Проценты голосов за кандидатов считаем от явки, берем число голосов за кандидата, делим на явку и домножаем на $100$. Проделаем это поэтапно. 

Сначала сохраним результат агрегирования в переменную `regs` и добавим новый столбец для явки в абсолютных значениях (в голосах).

In [ ]:
regs = d.groupby('region').agg('sum')

# новый столбец - сумма двух старых
regs["turnout"] = regs.invalid + regs.valid 

In [ ]:
regs.head(3)

Теперь добавим столбец с явкой в процентах:

In [ ]:
regs["turnout_perc"] = regs.turnout / regs.total * 100

In [ ]:
regs.head(3)

Осталось проделать аналогичные операции для голосов за разных кандидатов. Но повторять одно и то же пять раз не хочется (а что бы мы делали, если бы кандидатов было больше?). Давайте напишем функцию, которая будет принимать на вход столбец, делить все его значения на значения из столбца *turnout* и переводить все в проценты.

In [ ]:
def to_perc(x):
    return x / regs.turnout * 100

А теперь выберем из базы данных столбцы с голосами за кандидатов и применим к ним нашу функцию – воспользуемся методом `.apply()`.

In [ ]:
# axis = 0 - по столбцам, не по строкам 
perc = regs[['Zh' ,'Zu', 'Mi', 
             'Pr', 'Pu']].apply(to_perc, axis = 0) 

In [ ]:
perc.head(3)

Нужно переименовать столбцы в базе `perc`. Давайте сделаем это по-умному: возьмем названия столбцов в `perc` и приклеим к ним часть с `_perc`, чтобы названия столбцов с показателями в процентах отличались от показателей в абсолютных числах.

In [ ]:
# rename принимает на вход словарь пар
# старое название : новое название

perc = perc.rename(columns = {'Zh':'Zh_perc', 'Zu':'Zu_perc', 
                       'Mi':'Mi_perc', 'Pr':'Pr_perc', 
                       'Pu':'Pu_perc'})
perc.head()

Ура! Последний аккорд: соединим нашу таблицу `regs` с таблицей `perc`, чтобы все показатели были в одном месте. Способов объединять датафреймы много, но давайте обсудим их в следующий раз. А пока просто склеим две таблицы по столбцам с помощью метода `.concat()`.

In [ ]:
# axis = 1 - склеивание по столбцам
final = pd.concat([regs, perc], axis = 1) 

In [ ]:
final.head()

Приличную базу мы получили, можно перейти к чему-то более содержательному.

1. Вывести топ-5 регионов с максимальными  и минимальными процентами голосов за каждого кандидата